In [2]:
import torch
from transformers import RobertaTokenizerFast, RobertaForMaskedLM, AdamW
import os
import random

RuntimeError: Failed to import transformers.optimization because of the following error (look up to see its traceback):
Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:

# Load data from text files in the folder
def load_data_from_folder(folder_path):
    data = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            data.append(text)
    return data

In [ ]:
# Function to perform MLM on input tensor
def mlm(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)  # Masking 15% of tokens with value greater than 2 (special tokens)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero())
        tensor[i, selection] = 4  # Replace masked tokens with the <mask> token (token_id=4)
    return tensor



In [ ]:
# Load your local dataset (text files) from a folder
data_folder = 'path/to/your/folder'
text_data = load_data_from_folder(data_folder)


In [ ]:
# Initialize the tokenizer and create MLM input tensors
tokenizer = RobertaTokenizerFast.from_pretrained('TamBert')
random.shuffle(text_data)  # Shuffle the data for better training
input_ids = tokenizer(text_data, padding='max_length', truncation=True, return_tensors='pt')['input_ids']
input_ids_mlm = mlm(input_ids.detach().clone())


In [ ]:
# Prepare DataLoader
encodings = {'input_ids': input_ids_mlm, 'attention_mask': (input_ids > 0).int(), 'labels': input_ids}
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: tensor[idx] for key, tensor in self.encodings.items()}

dataset = Dataset(encodings)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


In [ ]:
# Initialize the model
model = RobertaForMaskedLM.from_pretrained('TamBert')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


In [ ]:
# Training loop
model.train()
optim = AdamW(model.parameters(), lr=1e-4)
epochs = 2

for epoch in range(epochs):
    for batch in loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()